Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [ ]:
from itertools import combinations
from collections import namedtuple, defaultdict
from random import choice
from copy import deepcopy

from tqdm.auto import tqdm
import numpy as np

import pickle
import os

from lab10 import *

## Basic Problem

### Training

In [ ]:
value_dictionary, hit_state = train_loop(0.001, 1)

In [ ]:
epsilon = 0.001
training_steps = 100_000

path = os.path.join("value_dictionaries", f"value_dictionary_{epsilon}_{training_steps}.pkl")

# if file exists, load it
try:
    with open(path, 'rb') as f:
        value_dictionary = pickle.load(f)
except FileNotFoundError:
    value_dictionary, hit_state = train_loop(epsilon, training_steps)
    with open(path, "wb") as f:
        pickle.dump(value_dictionary, f)

In [ ]:
res = play_game(agent_move, 
            random_move, 
            value_dictionary=value_dictionary,
            agent_player=1)

if res == "Player 1":
    print('Player 1 wins')
elif res == "Player 2":
    print('Player 2 wins')
else:
    print('Draw')

### Results

#### Random vs. Random

In [ ]:
player1 = random_move
player2 = random_move
agent_player = -1

print('Random player(p1) vs random player(p2)')
print()

results = evaluate(player1, 
                player2,
                value_dictionary=value_dictionary, 
                agent_player=agent_player,
                games=10_000)
print_results(results)

#### Agent vs. Random

In [ ]:
player1 = agent_move
player2 = random_move
agent_player = 1

print('Agent(p1) vs random player(p2)')
print()

results = evaluate(player1, 
                player2,
                value_dictionary=value_dictionary, 
                agent_player=agent_player,
                games=10_000)
print_results(results)

#### Random vs. agent

In [ ]:
player1 = random_move
player2 = agent_move
agent_player = 2

print('Random player(p1) vs agent(p2)')
print()

results = evaluate(player1, 
                player2, 
                value_dictionary=value_dictionary, 
                agent_player=agent_player,
                games=10_000)
print_results(results)

In [ ]:
player1 = random_move
player2 = agent_move
agent_player = 2

print('Random player(p1) vs agent(p2)')
print()

results = evaluate(player1, 
                player2, 
                value_dictionary=value_dictionary, 
                agent_player=agent_player,
                games=10_000, 
                trick=True)
print_results(results)

## Generalisation of the problem also on O 

### Training

In [ ]:
epsilon = 0.001
training_steps = 100_001

path = os.path.join("value_dictionaries", f"complete_value_dictionary_{epsilon}_{training_steps}.pkl")

# if file exists, load it
try:
    with open(path, 'rb') as f:
        complete_value_dictionary = pickle.load(f)
except FileNotFoundError:
    complete_value_dictionary, complete_hit_state = complete_train_loop(epsilon, training_steps)
    with open(path, "wb") as f:
        pickle.dump(complete_value_dictionary, f)

### Results

In [ ]:
player1 = complete_agent_move
player2 = random_move
agent_player = 1

print('Random player(p1) vs agent(p2)') if agent_player == 2 else print('Agent(p1) vs random player(p2)')
print()

results = complete_evaluate(player1, 
                        player2, 
                        complete_value_dictionary=complete_value_dictionary, 
                        agent_player=agent_player,
                        games=10_000)
print_results(results)

In [ ]:
player1 = random_move
player2 = complete_agent_move
agent_player = 2

print('Random player(p1) vs agent(p2)') if agent_player == 2 else print('Agent(p1) vs random player(p2)')
print()

results = complete_evaluate(player1, 
                        player2, 
                        complete_value_dictionary=complete_value_dictionary, 
                        agent_player=agent_player,
                        games=10_000)
print_results(results)

## Hyperparameter tuning

### Parameters Grid

In [ ]:
training_steps_list = [125, 250, 500, 1_000, 2_000, 3_000, 4_000, 5_000, 7_500, 10_000, 25_000, 50_000, 100_000, 250_000, 500_000, 1_000_000]

### Training Loops

In [ ]:
for training_steps in training_steps_list: 
    path = os.path.join("value_dictionaries", f"complete_value_dictionary_{epsilon}_{training_steps}.pkl")
    try:
        with open(path, 'rb') as f:
            complete_value_dictionary = pickle.load(f)
    except FileNotFoundError:
        complete_value_dictionary, complete_hit_state = complete_train_loop(epsilon, training_steps)
        with open(path, "wb") as f:
            pickle.dump(complete_value_dictionary, f)

### Results

Agent vs. Random

In [ ]:
# compare the different methods and see percentage of wins for each 

results = {}

for training_steps in training_steps_list: 
    path = os.path.join("value_dictionaries", f"complete_value_dictionary_{epsilon}_{training_steps}.pkl")
    with open(path, 'rb') as f:
        complete_value_dictionary = pickle.load(f)
        
    player1 = complete_agent_move
    player2 = random_move
    agent_player = 1
    result = complete_evaluate(player1, 
                            player2, 
                            complete_value_dictionary=complete_value_dictionary, 
                            agent_player=agent_player,
                            games=10_000)
    k, v = np.unique(result, return_counts=True)
    res = {key: value for key, value in zip(k, v)}
    results[training_steps] = res
        

In [ ]:
plt_results = defaultdict(dict)
for k in results.keys():
    plt_results[k]["Wins"] = results[k]["Player 1"] / 10_000
    plt_results[k]["Draws"] = results[k]["Draw"] / 10_000
    plt_results[k]["Losses"] = results[k]["Player 2"] / 10_000 if "Player 2" in results[k] else 0

In [ ]:
# print a table of the results

for k in plt_results.keys():
    print(f"Training steps: {k}")
    print(f"Wins: {plt_results[k]['Wins']}, Draws: {plt_results[k]['Draws']}, Losses: {plt_results[k]['Losses']}")
    print()

In [ ]:
# barplot of results, each value has a dictionary with win, losses and draws

import matplotlib.pyplot as plt

fig, ax = plt.subplots(4, 4, figsize=(20, 20))

for i, training_steps in enumerate(training_steps_list):
    col = i // 4
    row = i % 4
    ax[col, row].bar(plt_results[training_steps].keys(), plt_results[training_steps].values())
    ax[col, row].set_title(f"training steps: {training_steps}")
    ax[col, row].set_xlabel("Result")
    ax[col, row].set_ylabel("Count")
    ax[col, row].set_ylim(0, 1)
    ax[col, row].set_xticks([0, 1, 2])
    ax[col, row].set_xticklabels(["Win", "Draw", "Loss"])
    ax[col, row].grid(axis="y")
plt.tight_layout()
plt.show()

In [ ]:
# vector of numbers from 0 to 15 inclusive



In [ ]:
# plot number of wins, draws and losses for each training step 

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.plot(np.arange(len(plt_results.keys())), [plt_results[k]["Wins"] for k in plt_results.keys()], label="Wins")
ax.plot(np.arange(len(plt_results.keys())), [plt_results[k]["Draws"] for k in plt_results.keys()], label="Draws")
ax.plot(np.arange(len(plt_results.keys())), [plt_results[k]["Losses"] for k in plt_results.keys()], label="Losses")
ax.set_xlabel("Training steps")
ax.set_ylabel("Percentage")
ax.set_title("Percentage of wins, draws and losses for each training step")
ax.legend()
ax.grid()
plt.show()

Random vs. Agent

In [ ]:
# compare the different methods and see percentage of wins for each 

results = {}

for training_steps in training_steps_list: 
    path = os.path.join("value_dictionaries", f"complete_value_dictionary_{epsilon}_{training_steps}.pkl")
    with open(path, 'rb') as f:
        complete_value_dictionary = pickle.load(f)
        
    player1 = random_move
    player2 = complete_agent_move
    agent_player = 2
    result = complete_evaluate(player1, 
                            player2, 
                            complete_value_dictionary=complete_value_dictionary, 
                            agent_player=agent_player,
                            games=10_000)
    k, v = np.unique(result, return_counts=True)
    res = {key: value for key, value in zip(k, v)}
    results[training_steps] = res        

In [ ]:
plt_results = defaultdict(dict)
for k in results.keys():
    plt_results[k]["Wins"] = results[k]["Player 2"] / 10_000
    plt_results[k]["Draws"] = results[k]["Draw"] / 10_000
    plt_results[k]["Losses"] = results[k]["Player 1"] / 10_000 if "Player 1" in results[k] else 0

In [ ]:
# print a table of the results

for k in plt_results.keys():
    print(f"Training steps: {k}")
    print(f"Wins: {plt_results[k]['Wins']}, Draws: {plt_results[k]['Draws']}, Losses: {plt_results[k]['Losses']}")
    print()

In [ ]:
# barplot of results, each value has a dictionary with win, losses and draws

import matplotlib.pyplot as plt

fig, ax = plt.subplots(4, 4, figsize=(20, 20))

for i, training_steps in enumerate(training_steps_list):
    row = i // 4
    col = i % 4
    ax[row, col].bar(plt_results[training_steps].keys(), plt_results[training_steps].values())
    ax[row, col].set_title(f"training steps: {training_steps}")
    ax[row, col].set_xlabel("Result")
    ax[row, col].set_ylabel("Count")
    ax[row, col].set_ylim(0, 1)
    ax[row, col].set_xticks([0, 1, 2])
    ax[row, col].set_xticklabels(["Win", "Draw", "Loss"])
    ax[row, col].grid(axis="y")
plt.tight_layout()
plt.show()